# Creating a Collaborative Closing Transaction

In this section we'll create a Lightning Channel closing transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook:
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network Bolt #3: '[Funding Transaction Output](https://github.com/lightning/bolts/blob/master/03-transactions.md#funding-transaction-output)'

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/00-introduction.md)) defines the way to close a channel cooperativelly, lets focus on the legacy because most of the channels are closed using it.

    +-----------+                              +---------+
    |           |--(1)--  shutdown  ---------->|         |
    |           |<-(2)--  shutdown  -----------|         |
    |           |                              |         |
    |   Alice   |--(3)--  closing_signed  ---->|   Bob   |
    |           |<-(4)--  closing_signed  -----|         |
    |           |                              |         |
    |           |                              |         |
    |           |                              |         |
    +-----------+                              +---------+

Alice send the [`shutdown`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#closing-initiation-shutdown) message to Bob. This message initiate the closing process, along with the scriptpubkey it wants to be paid to:
* channel_id
* scriptpubkey

Once shutdown is complete, the channel is empty of HTLCs, there are no commitments for which a revocation is owed, and all updates are included on both commitments, the final current commitment transactions will have no HTLCs, and closing fee negotiation begins. The funder chooses a fee it thinks is fair, and signs the closing transaction with the scriptpubkey fields from the shutdown messages (along with its chosen fee) and sends the [`closing_signed`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#closing-negotiation-closing_signed) message; the other node then replies similarly, using a fee it thinks is fair. This exchange continues until both agree on the same fee or when one side fails the channel.

## The Legacy Closing Transaction

- version: 2
- locktime: 0
- txin count: 1
    - txin[0] outpoint: txid and output_index from funding_created message
    - txin[0] sequence: 0xFFFFFFFF
    - txin[0] script bytes: 0
    - txin[0] witness: 0 <signature_for_pubkey1> <signature_for_pubkey2>
- txout count: 1 or 2
    - txout amount: final balance to be paid to one node (minus fee_satoshis from closing_signed, if this peer funded the channel)
    - txout script: as specified in that node's scriptpubkey in its shutdown message

## Setup 
### Requirements
For this chapter we'll need the Bitcoin Core. This notebook has been tested with [v28.0](https://github.com/bitcoin/bitcoin/releases/tag/v28.0).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 2 - Commitment Transactions/First Commitment Transactions.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

In [2]:
# Determine our output scriptPubkeys and amounts (in satoshis)
# Alice pay for the fees
alice_output_value_sat = 3500000 - 500
alice_output_spk = bytes.fromhex("76a914") + hash160(bytes.fromhex(alice_funding_pubkey)) + bytes.fromhex("88ac")
bob_output_value_sat = 1500000
bob_output_spk = bytes.fromhex("76a914") + hash160(bytes.fromhex(bob_funding_pubkey)) + bytes.fromhex("88ac")

In [3]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Calculate the txid of the funding channel tx created on the previous notebook
channel_funding_txid = hash256(unsigned_channel_funding_tx)
# The index of the funding channel tx createt on the previous notebook
channel_funding_txindex = 1
# Convert index to bytes (little endian)
# Since the txid was already calculated, there is no need to invert the bytes to little-endian.
channel_funding_txindex = channel_funding_txindex.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    channel_funding_txid
    + channel_funding_txindex
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)
print("Inputs: " + inputs.hex())

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")
 
alice_output_value = alice_output_value_sat.to_bytes(8, byteorder="little", signed=True)

bob_output_value = bob_output_value_sat.to_bytes(8, byteorder="little", signed=True)

outputs = (
    alice_output_value
    + pushbytes(alice_output_spk)
    + bob_output_value
    + pushbytes(bob_output_spk)
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

Inputs: d739b5409647d162c84bf9b9d718bc7cfc79cb09558cfb13897d6e5032559fba0100000000ffffffff
unsigned_tx:  0200000001d739b5409647d162c84bf9b9d718bc7cfc79cb09558cfb13897d6e5032559fba0100000000ffffffff02ec653500000000001976a9145bd80a3647077ca7fd268dee2270a4442682ca1188ac60e31600000000001976a914b256d67c5f0170d88e6ff0dae453152e16df4de988ac00000000


In [4]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "fa8fc03fdb1ac33126ad085b7d87b77975b1b47879b68744d956386c0aba8997",
  "hash": "fa8fc03fdb1ac33126ad085b7d87b77975b1b47879b68744d956386c0aba8997",
  "version": 2,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "ba9f5532506e7d8913fb8c5509cb79fc7cbc18d7b9f94bc862d1479640b539d7",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.03499500",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 5bd80a3647077ca7fd268dee2270a4442682ca11 OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(motabE2LNUacCFkp2eaoTyzpJg1PQm8RUb)#7x6ps8y0",
        "hex": "76a9145bd80a3647077ca7fd268dee2270a4442682ca1188ac",
        "address": "motabE2LNUacCFkp2eaoTyzpJg1PQm8RUb",
        "type": "pubkeyhash"
      }
    },
    {
      "value": "0.01500000",
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 b25

In [5]:
scriptcode = redeemScript

# value of the funding channel output transaction (Output2)
value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(channel_funding_txid + channel_funding_txindex)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + channel_funding_txid
    + channel_funding_txindex
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

0200000059fba4daa75527b3fae9a44cfbe76fe287243459c9fd3e54d7d4d3adf2b4cbcf3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044d739b5409647d162c84bf9b9d718bc7cfc79cb09558cfb13897d6e5032559fba0100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c0000000000ffffffffb9bf0a970b99e17300fefe8083c196d7b461b7138eb8b019ee2eb70ad06d75650000000001000000


In [6]:
# SIGN ALICE COMMITMENT TRANSACTION
# Sign the sigHash with the input privkey1
# Create sigHash to be signed
tx_sighash = hash256(tx_digest_preimage)
alice_signing_key = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
alice_signature = alice_signing_key.sign_digest(tx_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
alice_signature = alice_signature + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
bob_signing_key = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
bob_signature = bob_signing_key.sign_digest(tx_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
bob_signature = bob_signature + bytes.fromhex("01")

witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(alice_signature)
    + alice_signature
    + varint_len(bob_signature)
    + bob_signature
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
signed_closing_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("Signed Closing Transaction: ",signed_closing_tx.hex())


Signed Closing Transaction:  02000000000101d739b5409647d162c84bf9b9d718bc7cfc79cb09558cfb13897d6e5032559fba0100000000ffffffff02ec653500000000001976a9145bd80a3647077ca7fd268dee2270a4442682ca1188ac60e31600000000001976a914b256d67c5f0170d88e6ff0dae453152e16df4de988ac0400483045022100f50af6a9419def83974ba5977575e6c013ae182f8190b8eb1b9c75fa5663310c0220797a9d586bb658877304f71e9e5a6bbd85e74883bfce0eba3f097b543f179fed0148304502210082f2fda779e179db07c306655cc13b9b54af363fa4cdb45fbed0b7827c935a6d02202263d9affd2acaf4337274ad7ad1e1807b0451bef20593db9312f84ae2457ee30147522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae00000000


In [7]:
print(node.testmempoolaccept(rawtxs=[signed_closing_tx.hex()]))

[{'txid': 'fa8fc03fdb1ac33126ad085b7d87b77975b1b47879b68744d956386c0aba8997', 'wtxid': '7b6c94aa4ffe90a2b8229dd7f607a2a4409f4f9f48dea414c23ae67bf539422d', 'allowed': True, 'vsize': 175, 'fees': {'base': Decimal('0.00000500'), 'effective-feerate': Decimal('0.00002857'), 'effective-includes': ['7b6c94aa4ffe90a2b8229dd7f607a2a4409f4f9f48dea414c23ae67bf539422d']}}]
